In [1]:
# Hungarian algorithm for maximum matching in bipartite graphs
# References:
# https://youtu.be/ory4WMX0rDU
# https://youtu.be/djaqebV3vmY
# R. Burkard et al, Linear Assignment Problems and Extensions 
# https://www.opt.math.tugraz.at/~cela/papers/lap_bericht.pdf

In [2]:
import numpy as np
from scipy.optimize import linear_sum_assignment
import timeit

In [29]:
def max_matching(W):
    nx, ny = W.shape
    M = [-1] * (nx + ny)   # matching  
    E = [[nx + k for k, w in enumerate(row) if w > 0] for j, row in enumerate(W)]  #edges
    
    # initialize
    P =[-1] * (nx + ny)   # predecessors
    T =[-1] * (nx + ny)   # R, T sets
    for x in xrange(nx):
        T[x] = 0 if M[x] == -1 else -1
    
    while True:    
        # take next unmarked element of R           
        x = next((j for j, m in enumerate(T[:nx]) if m == 0), None)
        
        # if there are no unmarked elements - terminate 
        if x is None:
            # output current matching and vertex cover
            M = [(x, M[x] - nx) for x in xrange(nx) if M[x] != -1]
            Q = ([j for j, r in enumerate(T[:nx]) if r == -1], [j for j, t in enumerate(T[nx:]) if t == 0])
            return M, Q
        
        else:
            # set mark on current vertex
            T[x] = 1 
            
            # scan all edges from x which are not part of matching
            for y in E[x]:
                if M[x] != y:
                    
                    # if vertex has not been visited
                    if T[y] == -1:
                        P[y] = x
                        T[y] = 0

                    # if vertex is unmatched, we have found an augmenting path
                    if M[y] == -1:
                        p = y
                        a = True
                        
                        # collect path by following predecessors and invert matching edges along the path
                        while p != -1:
                            if a is True:
                                M[p] = P[p]
                                M[P[p]] = p
                            p = P[p]
                            a = not a
                        
                        # reinitialize state
                        P =[-1] * (nx + ny)
                        T =[-1] * (nx + ny)
                        for j in xrange(nx):
                            T[j] = 0 if M[j] == -1 else -1
                            
                        break
                    
                    # if vertex is matched, continue to scan path
                    else:
                        x_ = M[y]
                        T[x_] = 0 if T[x_] == -1 else T[x_]
                        P[x_] = y

def max_weight_matching(W, u=None, v=None, fixed_assignment=None):
    EPS = 1.e-12
    nx, ny = W.shape
    
    # initialize potentials
    if nx > ny:
        u = np.zeros(nx) if u is None else u
        v = np.max(W, axis=0) if v is None else v
    else:
        u = np.max(W, axis=1) if u is None else u
        v = np.zeros(ny) if v is None else v
    
    # main loop
    while True:
        # compute reduced weights
        E = np.array([[u[j] + v[k] - w for k, w in enumerate(row)] for j, row in enumerate(W)])      
               
        # find tight subgraph, i.e add edges where E[i,j] == 0
        G = np.array([[1.0 if abs(e) < EPS else 0.0 for k, e in enumerate(row)] for j, row in enumerate(E)])
        
        if fixed_assignment is not None:
            G[fixed_assignment[0], :] = 0
            G[:, fixed_assignment[1]] = 0
            G[fixed_assignment[0], fixed_assignment[1]] = 1
        
        # find maximum matching and minimum vertex cover in tight subgraph
        M, Q = max_matching(G)
                
        # if matching is maximal: terminate
        if len(M) == min(nx, ny):
            break
        
        e = np.min([E[j, k] for k, e in enumerate(row) for j, row in enumerate(E) if (j not in Q[0] and k not in Q[1])])       
        
        if nx >= ny:
            u = [u[j] + e if j in Q[0] else u[j] for j in xrange(nx)]
            v = [v[k] - e if k not in Q[1] else v[k] for k in xrange(ny)]
        else:
            u = [u[j] - e if j not in Q[0] else u[j] for j in xrange(nx)]
            v = [v[k] + e if k in Q[1] else v[k] for k in xrange(ny)]
    
    # compute weight of maximum matching
    weight = np.sum(u) + np.sum(v)
    
    # output weight of maximum matching, list of matched nodes, potentials
    return weight, M, u, v


In [30]:
# Basic test
dims = [(5, 3), (3, 5), (1, 1), (10, 10)]

for d in dims:
    for iter in xrange(1000):
        W = np.random.randn(d[0], d[1])
        m_ = linear_sum_assignment(-W)
        cost_ = W[m_[0], m_[1]].sum()
        cost, matching, u, v = max_weight_matching(W)
        assert abs(cost - cost_) < 1.e-8

In [ ]:
N = 20
t1, t2, t3 = np.zeros(N), np.zeros(N), np.zeros(N)

for n in xrange(2, N):
    for j in xrange(100):
        W = np.random.randn(n, n)
        start_time = timeit.default_timer()
        cost, M, u, v = max_weight_matching(W)
        t1[n] += timeit.default_timer() - start_time

        start_time = timeit.default_timer()
        cost, M, _, _ = max_weight_matching(W[1:, 1:])
        t2[n] += timeit.default_timer() - start_time

        start_time = timeit.default_timer()
        cost, M, _, _ = max_weight_matching(W[1:, 1:], u[1:], v[1:])
        t3[n] += timeit.default_timer() - start_time

    print n, t1[n], t2[n], t3[n]

In [36]:
W = np.array([
    [0,       0,       0],       
    [0.12,    0.108,   0.096],   
    [0.05,    0.045,   0.04],    
    [0.014,   0.0126,  0.0112]      
])

m_ = linear_sum_assignment(-W)
cost_ = W[m_[0], m_[1]].sum()
cost, matching, u, v = max_weight_matching(W)

print cost_, cost

0.1762 0.1762
